In [1]:
import tensorflow as tf
import bz2
import os
import pickle
import sys
import scipy as sp

sys.path.append(os.path.abspath('../src'))
from fact_classification import *

2023-04-18 08:24:50.271313: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
df, df_crowdsourced, df_ground_truth = data_loading(local=True)
df['Sentiment'] = df.Sentiment.fillna(df.Sentiment[df.Verdict == -1].mean())

In [7]:
with bz2.open('../results/df_features.bz2') as f:
    X = pickle.load(f)

# Convert to compressed sparse row matrix
# X = sp.sparse.csr_matrix(df_features)

In [8]:
df_train, df_test, idx_train = test_train_split(df)

y = df['Verdict']
y_train = df_train['Verdict']
y_test = df_test['Verdict']

X_train = X[idx_train]
X_test = X[~idx_train]

# Column index for the numeric columns Sentiment and Length
col_idx_n = (df_features.columns == 'Sentiment') | (df_features.columns == 'Length')

# Column index for TF-IDF features on the raw Text column with n-grams=1
col_idx_w1 = df_features.columns.str.startswith('W1_')

# Column index for TF-IDF features on the raw Text column with n-grams=2
col_idx_w2 = df_features.columns.str.startswith('W2_')

# Column index for TF-IDF features on the stemmed text with n-grams=1
col_idx_ws = df_features.columns.str.startswith('WS_')

# Column index for POS features
col_idx_p = df_features.columns.str.startswith('P_')

# Column index for NER labels
col_idx_e = df_features.columns.str.startswith('E_')

In [12]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

# Load the dataset
url = "https://zenodo.org/record/3609356/files/crowdsourced.csv?download=1"
df = pd.read_csv(url)

# Split the dataset into training and testing sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Preprocessing: Tokenize and pad the sequences
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(train_df["Text"])

X_train = tokenizer.texts_to_sequences(train_df["Text"])
X_test = tokenizer.texts_to_sequences(test_df["Text"])

X_train = pad_sequences(X_train, maxlen=256, padding="post")
X_test = pad_sequences(X_test, maxlen=256, padding="post")

y_train = train_df["Verdict"]
y_test = test_df["Verdict"]

# Define the Keras LSTM model
def create_lstm_model():
    model = Sequential()
    model.add(Embedding(input_dim=10000, output_dim=128, input_length=256))
    model.add(LSTM(128, return_sequences=True))
    model.add(Dropout(0.5))
    model.add(LSTM(64))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation="sigmoid"))

    model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
    return model

# Instantiate and train the Keras LSTM classifier
lstm_classifier = KerasClassifier(build_fn=create_lstm_model, epochs=5, batch_size=64)
lstm_classifier.fit(X_train, y_train)

# Test the classifier
y_pred = lstm_classifier.predict(X_test)

# Evaluate the classifier
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

print("Classification Report:")
print(classification_report(y_test, y_pred))


/tmp/ipykernel_23154/4164159669.py:45: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  lstm_classifier = KerasClassifier(build_fn=create_lstm_model, epochs=5, batch_size=64)


Epoch 1/5
282/282 [==============================] - 288s 1s/step - loss: 0.6813 - accuracy: 0.1081
Epoch 2/5
282/282 [==============================] - 258s 914ms/step - loss: 0.6790 - accuracy: 0.1104
Epoch 3/5
282/282 [==============================] - 418s 1s/step - loss: 0.6791 - accuracy: 0.1057
Epoch 4/5
282/282 [==============================] - 475s 2s/step - loss: 0.6795 - accuracy: 0.1056
Epoch 5/5
141/141 [==============================] - 62s 428ms/step
Accuracy: 11.15%
Classification Report:
              precision    recall  f1-score   support

          -1       0.00      0.00      0.00      2926
           0       0.11      1.00      0.20       502
           1       0.00      0.00      0.00      1073

    accuracy                           0.11      4501
   macro avg       0.04      0.33      0.07      4501
weighted avg       0.01      0.11      0.02      4501



/home/tor/anaconda3/envs/fact_check/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/tor/anaconda3/envs/fact_check/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/tor/anaconda3/envs/fact_check/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(r

In [ ]:
def tf_classifier(x_features, y_labels, epochs=5, n_layers=1, n_neurons=10, verbose=0):
    if not isinstance(n_neurons, list) and isinstance(n_neurons, int):
        n_neurons = [n_neurons] * n_layers

    # Setup the layers
    model = tf.keras.Sequential()
    for n in range(n_layers):
        model.add(tf.keras.layers.Dense(n_neurons[n], activation='tanh', name=f'hidden{n+1}'))
    model.add(tf.keras.layers.Dense(3, activation='sigmoid'))

    # Compile the model
    model.compile(optimizer=tf.keras.optimizers.Adam(),
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                  metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

    # Train the model
    model.fit(x_features, y_labels, epochs=epochs, verbose=verbose)

    return model


In [ ]:
X = df_features.loc[idx_train, col_idx_w1 | col_idx_n].to_numpy()
y = df_train['Verdict'].to_numpy() + 1

In [ ]:
model = tf_classifier(X, y, epochs=5, n_layers=2, n_neurons=10, verbose=1)

In [ ]:
y_pred = model.predict(X)

In [ ]:
y_pred

In [ ]:
np.unique(y_pred.argmax(axis=1))

In [ ]:
np.unique(np.argmax(y_pred, axis=1))